# Image Processing Lab №5
Character Feature Extraction

In [4]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display

import matplotlib.pyplot as plt

from pathlib import Path

In [5]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [6]:
def open_image(image_name=None, images_dir=None):
    if images_dir:
        image_list = []
        for path in images_dir.iterdir():
            image = Image.open(path)
            image_list.append(image)
        return image_list

    image_folder = Path('input/'+image_name)
    try:
        image = Image.open(image_folder)
    except FileNotFoundError:
        print(f'Image file not found: {image_folder}')
    
    return image

pth = Path('input/')

In [7]:
def display_image(img):
    display(Image.fromarray(img.astype(np.uint8)))

In [8]:
def save_image(path: str, img: np.array):
    Image.fromarray(img).save(path)

In [9]:
def calculate_weight(img):
    return np.where(img == 0, True, False).sum()

In [10]:
def calculate_center_of_gravity(img_inp):
    img = np.where(img_inp == 0, 1, 0)
    x, y = 0, 0
    for i in range(img_inp.shape[0]):
        for j in range(img_inp.shape[1]):
            x += i * img[i, j]
            y += j * img[i, j]

    w = calculate_weight(img_inp)

    return x / w, y / w

In [11]:
def calculate_moments_of_inertia(img_inp, x_mean, y_mean):
    img = np.where(img_inp == 0, 1, 0)
    x, y = 0, 0
    for i in range(img_inp.shape[0]):
        for j in range(img_inp.shape[1]):
            x += np.power((j - y_mean), 2) * img[i, j]
            y += np.power((i - x_mean), 2) * img[i, j]
    
    return x, y

In [12]:
def save_projections(img, symbol):
    img_pr = np.where(img < 128, 1, 0)
    plt.figure(figsize=(15, 20))

    plt.subplot(2, 2, 4)
    plt.plot(range(img.shape[1]), img_pr.sum(axis=0), color='black')
    plt.title('Vertical Projection')
    plt.xlabel('Y')
    plt.ylabel('Weight')
    plt.ylim(0, img.shape[0])
    plt.xlim(0, img.shape[1])

    plt.subplot(2, 2, 2)
    plt.imshow(img, cmap='gray', aspect='auto')
    plt.title(f'Symbol')
    plt.axis('off')

    plt.subplot(2, 2, 1)
    plt.plot(img_pr.sum(axis=1), range(img.shape[0]-1, -1, -1), color='black')
    plt.title('Horizontal Projection')
    plt.xlabel('Weight')
    plt.ylabel('X')
    plt.ylim(0, img.shape[0])
    plt.xlim(0, img.shape[1])

    plt.tight_layout()
    plt.savefig(f'output/{symbol}_projections.png')

    plt.close()

In [13]:
stats = pd.DataFrame()

for i in range(len(alphabet)):
    img = np.array(open_image(alphabet[i]+'_preprocessed.png'))
    img = np.where(img < 128, 0, 255)

    M, N = img.shape[0], img.shape[1]

    statistics = {}
    x_q, y_q = M // 2, N // 2

    statistics['Symbol'] = alphabet[i]
    
    w = calculate_weight(img[:x_q, :y_q])
    statistics['Weight Quarter_1'] = w
    statistics['Weight Density Quarter_1'] = w / (x_q * y_q)

    w = calculate_weight(img[:x_q, y_q:])
    statistics['Weight Quarter_2'] = w
    statistics['Weight Density Quarter_2'] = w / (x_q * (N - y_q))

    w = calculate_weight(img[x_q:, :y_q])
    statistics['Weight Quarter_3'] = w
    statistics['Weight Density Quarter_3'] = w / ((M - x_q) * y_q)

    w = calculate_weight(img[x_q:, y_q:])
    statistics['Weight Quarter_4'] = w
    statistics['Weight Density Quarter_4'] = w / ((M - x_q) * (N - y_q))

    statistics['Center of Gravity coordinate_x'], statistics['Center of Gravity coordinate_y'] = \
        calculate_center_of_gravity(img)
    
    statistics['Relative Center of Gravity coordinate_x'], statistics['Relative Center of Gravity coordinate_y'] = \
        (statistics['Center of Gravity coordinate_x'] - 1) / (M - 1), (statistics['Center of Gravity coordinate_y'] - 1) / (N - 1)
    
    statistics['Axial Moment of Inertia_x'], statistics['Axial Moment of Inertia_y'] = \
        calculate_moments_of_inertia(img, statistics['Center of Gravity coordinate_x'], statistics['Center of Gravity coordinate_y'])
    
    statistics['Relative Axial Moment of Inertia_x'], statistics['Relative Axial Moment of Inertia_y'] = \
        statistics['Center of Gravity coordinate_x'] / (np.power(M, 2) * np.power(N, 2)), statistics['Center of Gravity coordinate_y'] / (np.power(M, 2) * np.power(N, 2))

    save_projections(img, alphabet[i])

    stats = pd.concat((stats, pd.DataFrame([statistics])), ignore_index=True)

In [14]:
stats.to_csv('output/statistics.csv', sep=',', index=False)